# 0- Donwload Unsloth

In [ ]:
# If you are in Kaggle
# %%capture
# !mamba install --force-reinstall aiohttp -y
# !pip install -U "xformers<0.0.26" --index-url https://download.pytorch.org/whl/cu121
# !pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

# # Temporary fix for https://github.com/huggingface/datasets/issues/6753
# !pip install datasets==2.16.0 fsspec==2023.10.0 gcsfs==2023.10.0

# import os
# os.environ["WANDB_DISABLED"] = "true"

In [8]:
# If you are in Colab
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

UsageError: Line magic function `%%capture` not found.


# 1- Importation des bibliotheque necessaire

In [1]:
from flask import Flask, request, jsonify, session 
from flask_cors import CORS
import ast

#### on va utiliser flask comme un framework de backend 

In [2]:
app = Flask(__name__)
CORS(app)

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

restaurants_aspect_model, restaurants_aspect_tokenizer = FastLanguageModel.from_pretrained(
    model_name = "auragFouad/mistral7b_aspect_restaurants",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(restaurants_aspect_model)

In [ ]:
laptops_aspect_model, laptops_aspect_tokenizer = FastLanguageModel.from_pretrained(
    model_name = "auragFouad/mistral7b_aspect_laptops", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(laptops_aspect_model) # Enable native 2x faster inference

In [ ]:
restaurants_sentiment_model, restaurants_sentiment_tokenizer = FastLanguageModel.from_pretrained(
    model_name = "auragFouad/llama3_sentiment_restaurants", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(restaurants_sentiment_model) # Enable native 2x faster inference

In [ ]:
laptops_sentiment_model, laptops_sentiment_tokenizer = FastLanguageModel.from_pretrained(
    model_name = "auragFouad/llama3_sentiment_laptops", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(laptops_sentiment_model) 

# Now we define the alpaca prompt and instructions for aspect prediction and sentiment classification on Restaurants and Laptops domains

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
restaurants_aspect_instruction_prompt = "Recognize all aspect terms in the given review delimited by triple quotes. The aspect terms are nouns or phrases appearing in the review that indicate specific aspects or features of the product/service. choose from the following aspects [RESTAURANT#GENERAL,SERVICE#GENERAL,FOOD#QUALITY,FOOD#STYLE_OPTIONS,DRINKS#STYLE_OPTIONS,DRINKS#PRICES,RESTAURANT#PRICES,RESTAURANT#MISCELLANEOUS,AMBIENCE#GENERAL,FOOD#PRICES,LOCATION#GENERAL,DRINKS#QUALITY]. Answer in the format [aspect] without any explanation, in case of multiple aspects append them into the list of aspects as a list like this [aspect_1, aspect_2]. If no aspect term exists, then only answer []."

In [ ]:
laptops_aspect_instruction_prompt = "Recognize all aspect terms in the given review delimited by triple quotes. The aspect terms are nouns or phrases appearing in the review that indicate specific aspects or features of the product/service. choose from the following aspects [RESTAURANT#GENERAL,SERVICE#GENERAL,FOOD#QUALITY,FOOD#STYLE_OPTIONS,DRINKS#STYLE_OPTIONS,DRINKS#PRICES,RESTAURANT#PRICES,RESTAURANT#MISCELLANEOUS,AMBIENCE#GENERAL,FOOD#PRICES,LOCATION#GENERAL,DRINKS#QUALITY]. Answer in the format [aspect] without any explanation, in case of multiple aspects append them into the list of aspects as a list like this [aspect_1, aspect_2]. If no aspect term exists, then only answer []." 

In [ ]:
restaurants_sentiment_instruction_prompt = "Given a sentence delimited by triple quotes between \"\" with a specific aspect next to it in between \"\", predict the sentiment polarity (positive, negative, or neutral) associated with that aspect in the given sentence. Answer back with the sentiment without any explanation.", # instruction

In [ ]:
laptops_sentiment_instruction_prompt = "Given a sentence delimited by triple quotes between \"\" with a specific aspect next to it in between \"\", predict the sentiment polarity (positive, negative, or neutral) associated with that aspect in the given sentence. Answer back with the sentiment without any explanation.", # instruction

## Now  i have 4 models:

    - aspect_restaurants_mistral_7b
    - aspect_laptop_mistral_7b
    - sentiment_restaurants_llama3_8b
    - sentiment_laptop_llama3_8b
    
For each task/domain we define a function for inference, and another for preprocessing the result

# Aspect/Restaurants

In [ ]:
# Inference with restaurants_aspect_model/restaurants_aspect_tokenizer/restaurants_aspect_instruction_prompt
def get_aspect_restaurants(user_description):
    input_text = user_description
    inputs = restaurants_aspect_tokenizer(
    [
      alpaca_prompt.format(
          restaurants_aspect_instruction_prompt,
          input_text, # input
          "", # output - leave this blank for generation!
      )
    ], return_tensors = "pt").to("cuda")

    outputs = laptops_aspect_model.generate(**inputs, max_new_tokens = 64, use_cache = True)
    output = restaurants_aspect_tokenizer.batch_decode(outputs)
    return output

# Preprocessing the model output
def get_aspect_restaurants_list(output):
    
    prediction = output
    prediction = prediction[0]
    prediction = prediction.split("### Response:\n")
    prediction = prediction[1]
    prediction = prediction.split("<|end_of_text|>")
    prediction = prediction[0]
    sides = prediction.split("]")
    prediction = sides[0]
    if prediction.endswith("'"):
        prediction = ast.literal_eval(prediction + "]")
    else:
        prediction = prediction + "']"
        prediction = ast.literal_eval(prediction)
    
    return prediction

# Aspect/Laptops

In [ ]:
# Inference with laptops_aspect_model/laptops_aspect_tokenizer/laptops_aspect_instruction_prompt
def get_aspect_laptops(user_description):
    input_text = user_description
    inputs = laptops_aspect_tokenizer(
    [
      alpaca_prompt.format(
          laptops_aspect_instruction_prompt,
          input_text, # input
          "", # output - leave this blank for generation!
      )
    ], return_tensors = "pt").to("cuda")

    outputs = laptops_aspect_model.generate(**inputs, max_new_tokens = 64, use_cache = True)
    output = laptops_aspect_tokenizer.batch_decode(outputs)
    
    return output

# Preprocessing the model output
def get_aspect_laptops_list(output):
    
    prediction = output
    # prediction = ast.literal_eval(prediction)
    prediction = prediction[0]
    prediction = prediction.split("### Response:\n")
    prediction = prediction[1]
    prediction = prediction.split("<|end_of_text|>")
    prediction = prediction[0]
    sides = prediction.split("]")
    prediction = sides[0]
    if prediction.endswith("'"):
        prediction = ast.literal_eval(prediction + "]")
    else:
        prediction = prediction + "']"
        prediction = ast.literal_eval(prediction)
        
    return prediction

# Sentiment/Restaurants

In [ ]:
# Get users prediction.
def get_aspect_sentiment_restaurants(output, aspect):
    aspect_sentiment_list = []
    predictions_list = []

    prediction = output
    prediction = prediction[0]
    prediction = prediction.split("### Response:\n")
    prediction = prediction[1]
    prediction = prediction.split("<|end_of_text|>")
    prediction = prediction[0]

    if "," in prediction:
        predictions = prediction.split(",")
        for prediction in predictions:
            predictions_list.append(prediction)
    else:
        predictions_list.append(prediction)
        
    for i in range(len(predictions_list)):
        aspect_sentiment_prediction = (aspect[i], predictions_list[i])
        aspect_sentiment_list.append(aspect_sentiment_prediction)
        
    return aspect_sentiment_list
        

# Inference with restaurants_sentiment_model/restaurants_sentiment_tokenizer/restaurants_sentiment_instruction_prompt
def get_sentiment_restaurants(user_description, restaurants_aspects_list):
    sentence = user_description
    aspect = restaurants_aspects_list
    
    input_text = f"The sentence is: \"{sentence}\" and the Aspect is: \"{aspect}\""

    inputs = restaurants_sentiment_tokenizer(
    [
      alpaca_prompt.format(
          restaurants_sentiment_instruction_prompt,
          input_text, # input
          "", # output - leave this blank for generation!
      )
    ], return_tensors = "pt").to("cuda")

    outputs = restaurants_sentiment_model.generate(**inputs, max_new_tokens = 64, use_cache = True)
    output = restaurants_sentiment_tokenizer.batch_decode(outputs)
    
    aspect_sentiment_restaurants = get_aspect_sentiment_restaurants(output, restaurants_aspects_list)
    
    return aspect_sentiment_restaurants

# Sentiment/Laptop

In [ ]:
# Get users prediction.
def get_aspect_sentiment_laptops(output, aspect):
    aspect_sentiment_list = []
    predictions_list = []

    prediction = output
    prediction = prediction[0]
    prediction = prediction.split("### Response:\n")
    prediction = prediction[1]
    prediction = prediction.split("<|end_of_text|>")
    prediction = prediction[0]

    if "," in prediction:
        predictions = prediction.split(",")
        for prediction in predictions:
            predictions_list.append(prediction)
    else:
        predictions_list.append(prediction)

    for i in range(len(predictions_list)):
        aspect_sentiment_prediction = (aspect[i], predictions_list[i])
        aspect_sentiment_list.append(aspect_sentiment_prediction)

    return aspect_sentiment_list
    
    
    
# Inference with laptops_sentiment_model/laptops_sentiment_tokenizer/laptops_sentiment_instruction_prompt
def get_sentiment_laptops(user_description, laptops_aspects_list):
    
    sentence = user_description
    aspect = laptops_aspects_list
    
    input_text = f"The sentence is: \"{sentence}\" and the Aspect is: \"{aspect}\""

    inputs = laptops_sentiment_tokenizer(
    [
      alpaca_prompt.format(
          laptops_sentiment_instruction_prompt,
          input_text, # input
          "", # output - leave this blank for generation!
      )
    ], return_tensors = "pt").to("cuda")

    outputs = laptops_sentiment_model.generate(**inputs, max_new_tokens = 64, use_cache = True)
    output = laptops_sentiment_tokenizer.batch_decode(outputs)

    # output = pre_process_output(output)
    aspect_sentiment_laptops = get_aspect_sentiment_restaurants(output, aspect)
    
    return aspect_sentiment_laptops

# Now we defined the functon for our APIs

In [ ]:
def generate_aspects(domain,model, user_description):
    
    if domain == "restaurants":
        output = get_aspect_restaurants(user_description)
        
        list_aspects = get_aspect_restaurants_list(output)
    else:
        output = get_aspect_laptops(user_description)
        
        list_aspects = get_aspect_laptops_list(output)
        
    return list_aspects

In [ ]:
# get_sentiment_restaurants(user_description, restaurants_aspects_list)

def get_sentiments(domain, model, user_description, list_aspects):
    
    if domain == "restaurants":
        aspect_sentiment_list = get_sentiment_restaurants(user_description, list_aspects)
    else:
        aspect_sentiment_list = get_sentiment_laptops(user_description, list_aspects)
        
    return aspect_sentiment_list

In [6]:
@app.route('/get_aspects', methods=['POST'])
def get_aspects_api():
    
    data = request.get_json()

    domain = data.get('domain', None)
    model = data.get('model', None)
    user_description = data.get('user_description', None)
    
    list_aspects = generate_aspects(domain,model, user_description)
    
    list_aspects_sentiments = get_sentiments(domain,model, user_description, list_aspects)
    
    return_message = {
        "list_aspects_sentiments": list_aspects_sentiments
    }
    
    return jsonify(return_message)

In [7]:
if __name__ == '__main__':
    app.run(port=5200)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5200
Press CTRL+C to quit
127.0.0.1 - - [31/Aug/2024 19:58:57] "OPTIONS /get_aspects HTTP/1.1" 200 -
127.0.0.1 - - [31/Aug/2024 19:58:58] "POST /get_aspects HTTP/1.1" 200 -


aspect_laptop_mistral_7b
aspect_laptop_mistral_7b


127.0.0.1 - - [31/Aug/2024 20:00:14] "OPTIONS /get_aspects HTTP/1.1" 200 -
127.0.0.1 - - [31/Aug/2024 20:00:14] "POST /get_aspects HTTP/1.1" 200 -


aspect_laptop_mistral_7b
aspect_laptop_mistral_7b


127.0.0.1 - - [31/Aug/2024 20:00:27] "OPTIONS /get_aspects HTTP/1.1" 200 -
127.0.0.1 - - [31/Aug/2024 20:00:27] "POST /get_aspects HTTP/1.1" 200 -


aspect_laptop_mistral_7b
aspect_laptop_mistral_7b


127.0.0.1 - - [31/Aug/2024 20:00:34] "OPTIONS /get_aspects HTTP/1.1" 200 -
127.0.0.1 - - [31/Aug/2024 20:00:34] "POST /get_aspects HTTP/1.1" 200 -


aspect_restaurants_mistral_7b
aspect_restaurants_mistral_7b
